In [99]:
import zipfile

import folium
import geopandas as gpd
import jageocoder
from lxml import etree
import pandas as pd

In [100]:
jageocoder.init()

In [101]:
# メンテナンスエリア
df0 = pd.read_json('https://network.mobile.rakuten.co.jp/assets/json/maintenance-short.json')
df1 = df0.query('都道府県 == "長野"').copy().reset_index(drop=True)
df1["住所"] = df1["対象地域"].str.replace("の一部地域", "")
df1

,都道府県,短期メンテナンス工事,作業開始時間,作業終了時間,お客様への影響,対象地域,住所
0,長野,,2022年6月21日 1:00,2022年6月21日 5:00,作業時間内で30分未満の通信断が発生し、音声通話およびデータ通信が、ご利用いただけない場合が...,長野県上松町大字上松の一部地域,長野県上松町大字上松
1,長野,,2022年6月21日 1:00,2022年6月21日 5:00,作業時間内で30分未満の通信断が発生し、音声通話およびデータ通信が、ご利用いただけない場合が...,長野県上松町大字小川の一部地域,長野県上松町大字小川
2,長野,,2022年6月21日 1:00,2022年6月21日 5:00,作業時間内で30分未満の通信断が発生し、音声通話およびデータ通信が、ご利用いただけない場合が...,長野県上松町大字荻原の一部地域,長野県上松町大字荻原
3,長野,,2022年6月21日 1:00,2022年6月21日 5:00,作業時間内で30分未満の通信断が発生し、音声通話およびデータ通信が、ご利用いただけない場合が...,長野県上松町緑町の一部地域,長野県上松町緑町
4,長野,,2022年6月16日 1:00,2022年6月16日 5:00,作業時間内で30分未満の通信断が発生し、音声通話およびデータ通信が、ご利用いただけない場合が...,長野県上田市上丸子字棒田の一部地域,長野県上田市上丸子字棒田
...,...,...,...,...,...,...,...
618,長野,,2022年6月23日 1:00,2022年6月23日 5:00,作業時間内で30分未満の通信断が発生し、音声通話およびデータ通信が、ご利用いただけない場合が...,長野県高山村大字高井字二反田の一部地域,長野県高山村大字高井字二反田
619,長野,,2022年6月21日 1:00,2022年6月21日 5:00,作業時間内で30分未満の通信断が発生し、音声通話およびデータ通信が、ご利用いただけない場合が...,長野県高森町下市田の一部地域,長野県高森町下市田
620,長野,,2022年6月16日 1:00,2022年6月16日 5:00,作業時間内で30分未満の通信断が発生し、音声通話およびデータ通信が、ご利用いただけない場合が...,長野県高森町山吹の一部地域,長野県高森町山吹
621,長野,,2022年6月21日 1:00,2022年6月21日 5:00,作業時間内で30分未満の通信断が発生し、音声通話およびデータ通信が、ご利用いただけない場合が...,長野県高森町山吹の一部地域,長野県高森町山吹


In [102]:
features = []

for s in df1["住所"]:

    results = jageocoder.searchNode(s)
 
    if len(results) > 0:

        node = results[0].node

        features.append(node.as_geojson())


In [103]:
pt_df = gpd.GeoDataFrame().from_features(features, crs="EPSG:4612")
pt_df

,geometry,id,name,level,priority,note,fullname
0,POINT (137.69202 35.80413),19178711,大字上松,5,2,aza_id:0000137/postcode:3995601,"[長野県, 木曽郡, 上松町, 大字上松]"
1,POINT (137.64275 35.76428),19178789,大字小川,5,2,aza_id:0004000/postcode:3995607,"[長野県, 木曽郡, 上松町, 大字小川]"
2,POINT (137.72983 35.75325),19179162,大字荻原,5,2,aza_id:0000105/postcode:3995608,"[長野県, 木曽郡, 上松町, 大字荻原]"
3,POINT (137.69555 35.78189),19179521,緑町,5,2,,"[長野県, 木曽郡, 上松町, 緑町]"
4,POINT (138.27739 36.31917),19190108,上丸子,5,2,aza_id:0017000/postcode:3860404,"[長野県, 上田市, 上丸子]"
...,...,...,...,...,...,...,...
618,POINT (138.36293 36.66421),19133967,大字高井,5,2,aza_id:0002000,"[長野県, 上高井郡, 高山村, 大字高井]"
619,POINT (137.87495 35.54461),19137079,下市田,5,2,aza_id:0005000/postcode:3993103,"[長野県, 下伊那郡, 高森町, 下市田]"
620,POINT (137.88259 35.58361),19139189,山吹,5,2,aza_id:0006000/postcode:3993101,"[長野県, 下伊那郡, 高森町, 山吹]"
621,POINT (137.88259 35.58361),19139189,山吹,5,2,aza_id:0006000/postcode:3993101,"[長野県, 下伊那郡, 高森町, 山吹]"


In [104]:
pt_df['address'] = pt_df['fullname'].apply(lambda x: "".join(x))
pt_df

,geometry,id,name,level,priority,note,fullname,address
0,POINT (137.69202 35.80413),19178711,大字上松,5,2,aza_id:0000137/postcode:3995601,"[長野県, 木曽郡, 上松町, 大字上松]",長野県木曽郡上松町大字上松
1,POINT (137.64275 35.76428),19178789,大字小川,5,2,aza_id:0004000/postcode:3995607,"[長野県, 木曽郡, 上松町, 大字小川]",長野県木曽郡上松町大字小川
2,POINT (137.72983 35.75325),19179162,大字荻原,5,2,aza_id:0000105/postcode:3995608,"[長野県, 木曽郡, 上松町, 大字荻原]",長野県木曽郡上松町大字荻原
3,POINT (137.69555 35.78189),19179521,緑町,5,2,,"[長野県, 木曽郡, 上松町, 緑町]",長野県木曽郡上松町緑町
4,POINT (138.27739 36.31917),19190108,上丸子,5,2,aza_id:0017000/postcode:3860404,"[長野県, 上田市, 上丸子]",長野県上田市上丸子
...,...,...,...,...,...,...,...,...
618,POINT (138.36293 36.66421),19133967,大字高井,5,2,aza_id:0002000,"[長野県, 上高井郡, 高山村, 大字高井]",長野県上高井郡高山村大字高井
619,POINT (137.87495 35.54461),19137079,下市田,5,2,aza_id:0005000/postcode:3993103,"[長野県, 下伊那郡, 高森町, 下市田]",長野県下伊那郡高森町下市田
620,POINT (137.88259 35.58361),19139189,山吹,5,2,aza_id:0006000/postcode:3993101,"[長野県, 下伊那郡, 高森町, 山吹]",長野県下伊那郡高森町山吹
621,POINT (137.88259 35.58361),19139189,山吹,5,2,aza_id:0006000/postcode:3993101,"[長野県, 下伊那郡, 高森町, 山吹]",長野県下伊那郡高森町山吹


In [105]:
gdf0 = gpd.read_file('A002005212015DDSWC20.zip').fillna('')
gdf0['address'] = gdf0['PREF_NAME'] + gdf0['CITY_NAME'] + gdf0['S_NAME']	
gdf0

,KEY_CODE,PREF,CITY,S_AREA,PREF_NAME,CITY_NAME,S_NAME,KIGO_E,HCODE,AREA,...,KIGO_I,MOJI,KBSUM,JINKO,SETAI,X_CODE,Y_CODE,KCODE1,geometry,address
0,202010010,20,201,001000,長野県,長野市,大字茂菅,,8101,1.224399e+06,...,,大字茂菅,8,260,98,138.16544,36.66410,0010-00,"POLYGON ((138.17024 36.66239, 138.17028 36.662...",長野県長野市大字茂菅
1,202010020,20,201,002000,長野県,長野市,大字西長野,,8101,1.031542e+06,...,,大字西長野,41,1765,866,138.17562,36.66252,0020-00,"POLYGON ((138.17914 36.65587, 138.17905 36.655...",長野県長野市大字西長野
2,20201003001,20,201,003001,長野県,長野市,大字長野,,8101,5.184155e+05,...,,大字長野,79,2352,1183,138.18366,36.65998,0030-01,"POLYGON ((138.18634 36.65456, 138.18631 36.654...",長野県長野市大字長野
3,20201003002,20,201,003002,長野県,長野市,大字長野,E1,8101,3.651206e+05,...,,大字長野,34,1198,563,138.18924,36.65885,0030-02,"POLYGON ((138.18972 36.65580, 138.18968 36.655...",長野県長野市大字長野
4,20201003002,20,201,003002,長野県,長野市,大字長野,E2,8101,2.345391e+06,...,,大字長野,0,0,0,138.18261,36.67680,0030-02,"POLYGON ((138.18619 36.66931, 138.18583 36.669...",長野県長野市大字長野
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3603,206020010,20,602,001000,長野県,栄村,大字豊栄,,8101,1.111009e+07,...,,大字豊栄,6,385,120,138.51241,36.99873,0010-00,"POLYGON ((138.54176 36.99406, 138.54152 36.993...",長野県栄村大字豊栄
3604,206020020,20,602,002000,長野県,栄村,大字北信,,8101,1.653190e+07,...,,大字北信,6,456,185,138.55190,36.99889,0020-00,"POLYGON ((138.57727 36.99493, 138.57803 36.994...",長野県栄村大字北信
3605,20602003001,20,602,003001,長野県,栄村,大字堺西部,,8101,3.539226e+07,...,,大字堺西部,6,444,171,138.52753,36.94829,0030-01,"POLYGON ((138.56011 36.95482, 138.56044 36.954...",長野県栄村大字堺西部
3606,20602003002,20,602,003002,長野県,栄村,大字堺東部,,8101,8.928413e+07,...,,大字堺東部,10,440,189,138.55139,36.91151,0030-02,"POLYGON ((138.59041 36.92028, 138.59024 36.919...",長野県栄村大字堺東部


In [106]:
# 空間結合
spj = gpd.sjoin(gdf0, pt_df).reset_index(drop=True)
spj

,KEY_CODE,PREF,CITY,S_AREA,PREF_NAME,CITY_NAME,S_NAME,KIGO_E,HCODE,AREA,...,geometry,address_left,index_right,id,name,level,priority,note,fullname,address_right
0,202010010,20,201,001000,長野県,長野市,大字茂菅,,8101,1.224399e+06,...,"POLYGON ((138.17024 36.66239, 138.17028 36.662...",長野県長野市大字茂菅,502,19661620,大字茂菅,5,2,aza_id:0000842/postcode:3800874,"[長野県, 長野市, 大字茂菅]",長野県長野市大字茂菅
1,202010010,20,201,001000,長野県,長野市,大字茂菅,,8101,1.224399e+06,...,"POLYGON ((138.17024 36.66239, 138.17028 36.662...",長野県長野市大字茂菅,501,19661620,大字茂菅,5,2,aza_id:0000842/postcode:3800874,"[長野県, 長野市, 大字茂菅]",長野県長野市大字茂菅
2,202010020,20,201,002000,長野県,長野市,大字西長野,,8101,1.031542e+06,...,"POLYGON ((138.17914 36.65587, 138.17905 36.655...",長野県長野市大字西長野,503,19661936,大字西長野,5,2,aza_id:0232000/postcode:3800871,"[長野県, 長野市, 大字西長野]",長野県長野市大字西長野
3,202010020,20,201,002000,長野県,長野市,大字西長野,,8101,1.031542e+06,...,"POLYGON ((138.17914 36.65587, 138.17905 36.655...",長野県長野市大字西長野,504,19661936,大字西長野,5,2,aza_id:0232000/postcode:3800871,"[長野県, 長野市, 大字西長野]",長野県長野市大字西長野
4,20201003001,20,201,003001,長野県,長野市,大字長野,,8101,5.184155e+05,...,"POLYGON ((138.18634 36.65456, 138.18631 36.654...",長野県長野市大字長野,507,19663685,横沢町,6,2,None,"[長野県, 長野市, 大字長野, 横沢町]",長野県長野市大字長野横沢町
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,205830040,20,583,004000,長野県,信濃町,大字柏原,,8101,3.541213e+07,...,"POLYGON ((138.20614 36.80230, 138.20619 36.802...",長野県信濃町大字柏原,174,19121761,大字柏原,5,2,aza_id:0000146/postcode:3891305,"[長野県, 上水内郡, 信濃町, 大字柏原]",長野県上水内郡信濃町大字柏原
619,205830040,20,583,004000,長野県,信濃町,大字柏原,,8101,3.541213e+07,...,"POLYGON ((138.20614 36.80230, 138.20619 36.802...",長野県信濃町大字柏原,175,19121761,大字柏原,5,2,aza_id:0000146/postcode:3891305,"[長野県, 上水内郡, 信濃町, 大字柏原]",長野県上水内郡信濃町大字柏原
620,205830050,20,583,005000,長野県,信濃町,大字古間,,8101,2.632251e+06,...,"POLYGON ((138.22908 36.79441, 138.22908 36.794...",長野県信濃町大字古間,171,19120501,大字古間,5,2,aza_id:0000111/postcode:3891313,"[長野県, 上水内郡, 信濃町, 大字古間]",長野県上水内郡信濃町大字古間
621,205830060,20,583,006000,長野県,信濃町,大字富濃,,8101,1.132806e+07,...,"POLYGON ((138.25265 36.80212, 138.25302 36.801...",長野県信濃町大字富濃,172,19121386,大字富濃,5,2,aza_id:0005000/postcode:3891312,"[長野県, 上水内郡, 信濃町, 大字富濃]",長野県上水内郡信濃町大字富濃


In [107]:
with zipfile.ZipFile("rakuten.kmz") as existing_zip:
    existing_zip.extractall("kmz")

ns = {"kml": "http://www.opengis.net/kml/2.2"}
tree = etree.parse("./kmz/doc.kml")

data = []

for i in tree.xpath("//kml:Placemark", namespaces=ns):

    d = {}

    d["名前"] = "".join(i.xpath("./kml:name/text()", namespaces=ns)).strip()
    d["geometry"] = "".join(
        i.xpath("./kml:Point/kml:coordinates/text()", namespaces=ns)
    ).strip()

    for j in i.xpath("./kml:ExtendedData/kml:Data", namespaces=ns):

        k = j.attrib["name"].strip()
        v = "".join(j.xpath("./kml:value/text()", namespaces=ns)).strip()

        d[k] = v

    data.append(d)

df_map_1 = pd.DataFrame(data)

df_map_1[["経度", "緯度", "標高"]] = df_map_1["geometry"].str.split(",", expand=True)

# 文字から数値に変換
df_map_1["経度"] = pd.to_numeric(df_map_1["経度"])
df_map_1["経度"] = pd.to_numeric(df_map_1["経度"])

# df_map.columns
# df_map['開局状態'].unique() # ['開局', '未開局', nan]
df_map_2 = df_map_1.query('開局状態 == "開局" | 開局状態 == "未開局"')

df_map_3 = df_map_2[['eNB-LCID', '住所', '市区町村', '開局状態', '経度', '緯度']].copy()

df_map_3

,eNB-LCID,住所,市区町村,開局状態,経度,緯度
0,"212995-1,2,3",長野県塩尻市広丘郷原１７６２−２８８,塩尻市,開局,137.932062,36.119826
1,"212995-10,11,12",長野県塩尻市大門八番町９−１,塩尻市,開局,137.948550,36.11327
2,"212995-4,5,6",長野県塩尻市大門９５−１１,塩尻市,開局,137.946973,36.123974
3,"212995-7,8,9",長野県塩尻市桔梗ケ原７１−３６９,塩尻市,開局,137.942787,36.11194
4,"212997-1,2,3",長野県松本市内田４５−１,松本市,開局,137.983550,36.17247
...,...,...,...,...,...,...
193,213198-3,長野県大町市平,大町市,開局,137.830120,36.51402
194,"213198-7,8",長野県大町市平,大町市,開局,137.812290,36.52665
195,213200-10,長野県安曇野市穂高有明,安曇野市,開局,137.861770,36.35479
196,"213201-10,11,12",長野県北安曇郡松川村,北安曇郡松川村,開局,137.867930,36.40412


In [108]:
# 行政区域データ
gdf_area = gpd.read_file('N03-20210101_20_GML.zip!N03-20210101_20_GML/N03-21_20_210101.shp').fillna('-')
gdf_chusin = gdf_area[gdf_area['N03_004'].isin(['松本市', '塩尻市', '安曇野市', '大町市', '麻績村', '生坂村', '山形村', '朝日村', '筑北村', '池田町', '松川村', '白馬村', '小谷村', '上松町', '南木曽町', '木祖村', '王滝村','大桑村', '木曽町'])]
gdf_chusin

,N03_001,N03_002,N03_003,N03_004,N03_007,geometry
1,長野県,-,-,松本市,20202,"POLYGON ((138.05548 36.37721, 138.05549 36.377..."
11,長野県,-,-,大町市,20212,"POLYGON ((137.76860 36.65856, 137.76866 36.658..."
14,長野県,-,-,塩尻市,20215,"POLYGON ((137.96450 36.17149, 137.96475 36.171..."
19,長野県,-,-,安曇野市,20220,"POLYGON ((137.72955 36.42532, 137.73017 36.425..."
54,長野県,-,木曽郡,上松町,20422,"POLYGON ((137.72809 35.82215, 137.72833 35.822..."
55,長野県,-,木曽郡,南木曽町,20423,"POLYGON ((137.53581 35.69008, 137.53597 35.690..."
56,長野県,-,木曽郡,木祖村,20425,"POLYGON ((137.77616 36.07500, 137.77620 36.074..."
57,長野県,-,木曽郡,王滝村,20429,"POLYGON ((137.43761 35.90505, 137.43766 35.903..."
58,長野県,-,木曽郡,大桑村,20430,"POLYGON ((137.81205 35.75964, 137.81192 35.759..."
59,長野県,-,木曽郡,木曽町,20432,"POLYGON ((137.63561 36.02590, 137.63574 36.025..."


In [109]:
# ベースマップ
map = folium.Map(
    tiles = None,
    location = [36.651278, 138.180944],
    zoom_start = 11,
    control_scale = True
)

# Googleマップ標準
folium.raster_layers.TileLayer(
    'https://{s}.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
    subdomains = ['mt0','mt1','mt2','mt3'],
    name = "Google Map",
    attr = "<a href='https://developers.google.com/maps/documentation' target='_blank'>Google Map</a>"
).add_to(map)

# 国土地理院(白地図)
folium.raster_layers.TileLayer(
    'https://cyberjapandata.gsi.go.jp/xyz/blank/{z}/{x}/{y}.png',
    name = "国土地理院(白地図)",
    attr = "<a href='https://maps.gsi.go.jp/development/ichiran.html'>国土地理院</a>"
).add_to(map)

cell_group = folium.FeatureGroup(name="基地局").add_to(map)

for i, r in df_map_3.iterrows():

    # popup
    # eNB-LCID, 住所, 市区町村, 開局状態	
    html = f'''
    <p>eNB-LCID: {r['eNB-LCID']}</p>
    <p>住所: {r['住所']}</p>
    <p>市区町村: {r['市区町村']}</p>
    <p>開局状態: {r['開局状態']}</p>
    '''

    cell_group.add_child(
        folium.Marker(
            location = [ r["緯度"], r["経度"] ],
            popup = folium.Popup(html=html, max_width=300),
            icon=folium.Icon(color = 'pink' if r['開局状態'] == '開局' else 'black')
        )
    )

# 楽天モバイルエリア4Gマップレイヤー(予定1)
folium.raster_layers.TileLayer(
    name="(4G)楽天モバイルエリア(予定1)",
    tiles='https://gateway-api.global.rakuten.com/dsd/geoserver/4g2m/mno_coverage_map/gwc/service/gmaps?LAYERS=mno_coverage_map:all_map&FORMAT=image/png&TRANSPARENT=TRUE&x={x}&y={y}&zoom={z}',
    fmt='image/png',
    attr="<a href='https://network.mobile.rakuten.co.jp/'>楽天モバイル</a>",
    tms=False,
    overlay=True,
    control=True,
    opacity=1.0,
    show=False,
).add_to(map)

# 楽天モバイルエリア4Gマップレイヤー(予定2)
folium.raster_layers.TileLayer(
    name="(4G)楽天モバイルエリア(予定2)",
    tiles='https://gateway-api.global.rakuten.com/dsd/geoserver/4g4m/mno_coverage_map/gwc/service/gmaps?LAYERS=mno_coverage_map:all_map&FORMAT=image/png&TRANSPARENT=TRUE&x={x}&y={y}&zoom={z}',
    fmt='image/png',
    attr="<a href='https://network.mobile.rakuten.co.jp/'>楽天モバイル</a>",
    tms=False,
    overlay=True,
    control=True,
    opacity=1.0,
    show=False,
).add_to(map)

# 楽天モバイルエリア5Gマップレイヤー
folium.raster_layers.TileLayer(
    name="(5G)楽天モバイルエリア",
    tiles='https://gateway-api.global.rakuten.com/dsd/geoserver/5g/mno_coverage_map/gwc/service/gmaps?LAYERS=mno_coverage_map:all_map&FORMAT=image/png&TRANSPARENT=TRUE&x={x}&y={y}&zoom={z}',
    fmt='image/png',
    attr="<a href='https://network.mobile.rakuten.co.jp/'>楽天モバイル</a>",
    tms=False,
    overlay=True,
    control=True,
    opacity=1.0,
    show=False,
).add_to(map)

# ポリゴン
folium.GeoJson(
    data = spj.to_json(),
    style_function = lambda x:{
        "fillColor": 'red',
        'fillOpacity': 0.7,
        "color": "black",
        "weight": 1
    },
    name = '短期メンテナンス工事エリア',
    show = True,
    tooltip = folium.features.GeoJsonTooltip(
        fields=['PREF_NAME', 'CITY_NAME', 'S_NAME' ],
        aliases=['都道府県名', '区町村名', '町丁・字等名称' ]
    )
).add_to(map)

# ポイント
folium.GeoJson(
    data = pt_df.to_json(),
    name = '短期メンテナンス工事エリア(ジオコーディングしたポイント)',
    show = True,
    tooltip = folium.features.GeoJsonTooltip(
        fields=['address', ],
        aliases=['住所', ]
    )
).add_to(map)

# 中信エリア
folium.features.GeoJson(
    data = gdf_chusin.to_json(),
    style_function = lambda x:{
        'fillColor': '#000000',
        'fillOpacity': 0,
        'color' : '#FF0000',
        'weight': 1.5
    },
    name = "中信エリア(出典:国土交通省)",
    show = False,
    popup = folium.features.GeoJsonPopup(
        fields = ['N03_003', 'N03_004'],
        aliases = ['郡・政令都市名', '市区町村名']
    ),
).add_to(map)

# レイヤーコントロール
folium.LayerControl().add_to(map)

map

In [ ]:
map.save('map.html')